# Creating an Einstein Toolkit thorn

### Now that we have coded of the Weyl scalars and associated invariants, we need to create a thorn to incorporate the generated C code into the Toolkit to calculate the scalars.

To create an Einstein Toolkit thorn to numerically compare the two codes, we will need a few more files, as required by the Einstein Toolkit: the scheduling, parameter, and interface $\text{.ccl files}$ (our specific thorn does not require the optional $\text{configuration.ccl}$ file). 

We will start with the file $\text{interface.ccl}$. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the scalars $\text{psi4r}$
and $\text{psi4i}$ to be visible to other thorns by using the keyword "public". 

Now that we have built the full form of both the real and imaginary components of all five Weyl scalars, we can generate C code to compute them. We will also set up the directory structure to build an Einstein Toolkit thorn. 

We first need to output the equations to a $\text{.h}$ file so that they can be used by the C code.

In [1]:
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm
from outputC import *
import sympy as sp
import NRPy_param_funcs as par
import loop

import WeylScal4NRPy.WeylScalars_Cartesian as weyl
weyl.WeylScalars_Cartesian()

par.set_parval_from_str("output_scalars","all_psis_and_invariants")
output_scalars = par.parval_from_str("output_scalars")

!mkdir WeylScal4NRPy     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir WeylScal4NRPy/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
# par.set_parval_from_str("outputC::SIMD_enable",True)
scalars_to_print = [lhrh(lhs=gri.gfaccess("out_gfs","psi4r"),rhs=weyl.psi4r_rhs),\
                    lhrh(lhs=gri.gfaccess("out_gfs","psi4i"),rhs=weyl.psi4i_rhs)]

get_position_pointers = "\tconst CCTK_REAL xx0 = x[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                       \n\tconst CCTK_REAL xx1 = y[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                       \n\tconst CCTK_REAL xx2 = z[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
\n"

get_psi_pointers = "\tconst double psi4r = psi4rGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi4i = psi4iGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi3r = psi3rGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi3i = psi3iGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi2r = psi2rGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi2i = psi2iGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi1r = psi1rGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi1i = psi1iGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi0r = psi0rGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
                  \n\tconst double psi0i = psi0iGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\
\n"

if output_scalars is "all_psis" or "all_psis_and_invariants":
    scalars_to_print = scalars_to_print + [\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi4r"),rhs=weyl.psi4r_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi4i"),rhs=weyl.psi4i_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi3r"),rhs=weyl.psi3r_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi3i"),rhs=weyl.psi3i_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi2r"),rhs=weyl.psi2r_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi2i"),rhs=weyl.psi2i_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi1r"),rhs=weyl.psi1r_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi1i"),rhs=weyl.psi1i_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi0r"),rhs=weyl.psi0r_rhs),\
                                           lhrh(lhs=gri.gfaccess("out_gfs","psi0i"),rhs=weyl.psi0i_rhs),]
    if output_scalars is "all_psis_and_invariants":
#        scalars_to_print = scalars_to_print + [\
        invars_to_print = [\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvIr"),rhs=weyl.curvIr_rhs),\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvIi"),rhs=weyl.curvIi_rhs),\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvJr"),rhs=weyl.curvJr_rhs),\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvJi"),rhs=weyl.curvJi_rhs),\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvJ1"),rhs=weyl.curvJ1_rhs),\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvJ2"),rhs=weyl.curvJ2_rhs),\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvJ3"),rhs=weyl.curvJ3_rhs),\
                           lhrh(lhs=gri.gfaccess("out_gfs","curvJ4"),rhs=weyl.curvJ4_rhs),]
        invars = fin.FD_outputC("returnstring",invars_to_print)
        invars_looped = loop.loop(["i2","i1","i0"],["2","2","2"],["cctk_lsh[2]-2","cctk_lsh[1]-2","cctk_lsh[0]-2"],\
                                  ["1","1","1"],["#pragma omp parallel for","",""],"",get_psi_pointers+invars)
        file = open("WeylScal4NRPy/src/WeylScal4NRPy_invars.h", "w")
        file.write(str(invars_looped))

psis = fin.FD_outputC("returnstring",scalars_to_print)
psis_looped = loop.loop(["i2","i1","i0"],["2","2","2"],["cctk_lsh[2]-2","cctk_lsh[1]-2","cctk_lsh[0]-2"],\
                        ["1","1","1"],["#pragma omp parallel for","",""],"",get_position_pointers+psis)
file = open("WeylScal4NRPy/src/WeylScal4NRPy_psis.h", "w")
file.write(str(psis_looped))


111225

In [2]:
%%writefile WeylScal4NRPy/interface.ccl

implements: WeylScal4NRPy

inherits:   admbase Boundary Grid methodoflines

public:
CCTK_REAL NRPyPsi4_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  psi4r, psi4i
} "Psi4_group"

public:
CCTK_REAL NRPyPsi3210_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  psi3r,psi3i,psi2r,psi2i,psi1r,psi1i,psi0r,psi0i
} "Psi3210_group"

public:
CCTK_REAL NRPyInvars_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvIr,NRPycurvIi,NRPycurvJr,NRPycurvJi,NRPycurvJ1,NRPycurvJ2,NRPycurvJ3,NRPycurvJ4
} "Psi3210_group"

Overwriting WeylScal4NRPy/interface.ccl


We will now write the file $\text{param.ccl}$. This file allows the listed parameters to be set at runtime. We also give allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). Here, the first two parameters tell the Method of Lines thorn that none of our variables need to be evolved in time. The last two determine how often we will calculate $\psi_4$ and how many timesteps to skip at the beginning; currently, they are set to calculate $\psi_4$ at every timestep, starting from the first in the simulation. 

In [3]:
%%writefile WeylScal4NRPy/param.ccl

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3

restricted:
CCTK_INT WeylScal4NRPy_calc_every "WeylScal4_psi4_calc_Nth_calc_every" STEERABLE=ALWAYS
{
  *:* :: ""
} 1

private:
CCTK_KEYWORD output_scalars "Whether to output all Weyl scalars, just psi4, or all scalars and invariants" 
{  
  "psi4_only" :: ""
  "all_psis" :: ""
  "all_psis_and_invariants" :: ""
} "psi4_only"


Overwriting WeylScal4NRPy/param.ccl


Finally, we will write the file $\text{schedule.ccl}$; its official documentation is found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). This file dictates when the various parts of the thorn will be run. We first assign storage for both the real and imaginary components of $\psi_4$, and then specify that we want our code run in  the $\text{MoL_PseudoEvolution}$ schedule group, after the ADM variables are set. At this step, we declare that we will be writing code in C. We also specify the gridfunctions that we wish to read in from memory--in our case, we need all the components of $K_{ij}$ and $g_{ij}$ (which we have been calling $\gamma_{ij}$) in addition to the coordinate values.

In [4]:
%%writefile WeylScal4NRPy/schedule.ccl

STORAGE: NRPyPsi4_group[timelevels]
if (CCTK_EQUALS(output_scalars, "all_psis_and_invariants") || CCTK_EQUALS(output_scalars, "all_psis"))
{
 STORAGE: NRPyPsi3210_group[timelevels]
}
if (CCTK_EQUALS(output_scalars, "all_psis_and_invariants"))
{
 STORAGE: NRPyInvars_group[timelevels]
}

schedule group WeylScal4NRPy_group in MoL_PseudoEvolution after ADMBase_SetADMVars
{
} "Schedule WeylScal4NRPy group"

schedule weylscal4_mainfunction in WeylScal4NRPy_group
{
  LANG: C
  READS: admbase::kxx(Everywhere)
  READS: admbase::kxy(Everywhere)
  READS: admbase::kxz(Everywhere)
  READS: admbase::kyy(Everywhere)
  READS: admbase::kyz(Everywhere)
  READS: admbase::kzz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::z(Everywhere)
  WRITES: WeylScal4::psi4i(Interior)
  WRITES: WeylScal4::psi4r(Interior)
  WRITES: WeylScal4::psi3i(Interior)
  WRITES: WeylScal4::psi3r(Interior)
  WRITES: WeylScal4::psi2i(Interior)
  WRITES: WeylScal4::psi2r(Interior)
  WRITES: WeylScal4::psi1i(Interior)
  WRITES: WeylScal4::psi1r(Interior)
  WRITES: WeylScal4::psi0i(Interior)
  WRITES: WeylScal4::psi0r(Interior)
  WRITES: WeylScal4::NRPycurvIi(Interior)
  WRITES: WeylScal4::NRPycurvIr(Interior)
  WRITES: WeylScal4::NRPycurvJi(Interior)
  WRITES: WeylScal4::NRPycurvJr(Interior)
  WRITES: WeylScal4::NRPycurvJ1(Interior)
  WRITES: WeylScal4::NRPycurvJ2(Interior)
  WRITES: WeylScal4::NRPycurvJ3(Interior)
  WRITES: WeylScal4::NRPycurvJ4(Interior)

} "Call WeylScal4NRPy main function"


Overwriting WeylScal4NRPy/schedule.ccl


In [5]:
#if (CCTK_EQUALS(output_all_psis,"true"))
#{
#    {
#  } /*"Allocate storage for the other psis if desired"*/
#    if (CCTK_EQUALS(output_invars,"true"))
#    {
#        {
#        } /*"Allocate storage for scalar invariants if desired"*/
#    }
#}

We will also need $\text{make.code.defn}$, which states which files need to be compiled. This thorn only has the one C file to compile.

In [6]:
%%writefile WeylScal4NRPy/src/make.code.defn

SRCS = WeylScal4NRPy.c

Overwriting WeylScal4NRPy/src/make.code.defn
